In [1]:
import firebase_admin
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from firebase_admin import credentials, firestore
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Firebase Initialization
cred = credentials.Certificate("adminkey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

# Constants
INCLUDE_ONLY = ['11111']
ACTIVITIES = ['sit', 'walk', 'upstairs']
TIME_START, TIME_END = 3000, 27000
SAMPLING_RATE = 100  # Hz
OVERLAP = 0.5
CHUNK_SIZE = 1  # of seconds
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 0.001
EMB_SIZE = 128
NHEAD = 8
NHID = 128
NLAYERS = 3
DROPOUT = 0.1

In [2]:
def fetch_c2st_data():
    data_before, data_after, labels_before, labels_after = [], [], [], []

    data_ref = db.collection("data")
    for treatment_doc in data_ref.stream():
        treatment_name = treatment_doc.id
        print(f"Treatment: {treatment_name}")

        for patient_id in INCLUDE_ONLY:
            print(f"  Patient: {patient_id}")
            subject_ref = data_ref.document(treatment_name).collection(patient_id)

            for activity in ACTIVITIES:
                activity_ref = subject_ref.document(activity)
                phone_location_col = activity_ref.collections()

                for phone_location in phone_location_col:
                    location_name = phone_location.id
                    print(f"    Activity: {activity}")
                    print(f"      Phone Location: {location_name}")

                    for recording in phone_location.stream():
                        record = recording.to_dict()
                        if 'acceleration' not in record:
                            continue

                        df = pd.DataFrame(record['acceleration'])
                        timestamps = df['timestamp']
                        min_time = timestamps.min()
                        max_time = timestamps.max()

                        time_start_cutoff = min_time + 2500
                        time_end_cutoff = max_time - 2500
                        df = df[(timestamps >= time_start_cutoff) & (timestamps <= time_end_cutoff)]
                        if df.empty:
                            continue

                        combined = pd.DataFrame({
                            'ax': df['accelerometer'].apply(lambda x: x['x']),
                            'ay': df['accelerometer'].apply(lambda x: x['y']),
                            'az': df['accelerometer'].apply(lambda x: x['z']),
                            'gx': df['gyroscope'].apply(lambda x: x['x']),
                            'gy': df['gyroscope'].apply(lambda x: x['y']),
                            'gz': df['gyroscope'].apply(lambda x: x['z']),
                        })

                        if treatment_name == 'BeforeTreatment':
                            data_before.append(combined)
                            labels_before.append(0)
                        elif treatment_name == 'AfterTreatment':
                            data_after.append(combined)
                            labels_after.append(1)

    return data_before, labels_before, data_after, labels_after

def prepare_c2st_chunks(data_raw, labels_raw, chunk_size_sec, sampling_rate, overlap=OVERLAP):
    data, labels = [], []
    chunk_samples = int(chunk_size_sec * sampling_rate)
    step = int(chunk_samples * (1 - overlap))

    for i, df in enumerate(data_raw):
        for start in range(0, len(df) - chunk_samples + 1, step):
            chunk = df.iloc[start:start + chunk_samples]
            if len(chunk) == chunk_samples:
                data.append(chunk.values)
                labels.append(labels_raw[i])
    return np.array(data), np.array(labels)

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout=0.1, maxlen=5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(maxlen, emb_size)
        position = torch.arange(0, maxlen, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_size, 2).float() * (-math.log(10000.0) / emb_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [4]:
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim=6, emb_size=EMB_SIZE, nhead=NHEAD, nhid=NHID, nlayers=NLAYERS, nclasses=2, dropout=DROPOUT):
        super().__init__()
        self.embedding = nn.Linear(input_dim, emb_size)
        self.pos_encoder = PositionalEncoding(emb_size, dropout)
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_size, nhead=nhead, dim_feedforward=nhid, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=nlayers)
        self.classifier = nn.Linear(emb_size, nclasses)
        self.emb_size = emb_size

    def forward(self, x):
        x = self.embedding(x) * math.sqrt(self.emb_size)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)
        x = self.classifier(x)
        return x

In [5]:
def train_c2st_classifier():
    print("Fetching Before and After data for C2ST...")
    data_before, labels_before, data_after, labels_after = fetch_c2st_data()

    X_before, y_before = prepare_c2st_chunks(data_before, labels_before, CHUNK_SIZE, SAMPLING_RATE, OVERLAP)
    X_after, y_after = prepare_c2st_chunks(data_after, labels_after, CHUNK_SIZE, SAMPLING_RATE, OVERLAP)

    print(f"Before data chunks: {X_before.shape}, After data chunks: {X_after.shape}")

    # Combine data
    X_train, y_train = X_before, y_before
    X_test, y_test = X_after, y_after

    # Convert to tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.long)

    train_ds = TensorDataset(X_train, y_train)
    test_ds = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TransformerClassifier(nclasses=2).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0, 0, 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            correct += (preds.argmax(dim=1) == yb).sum().item()
            total += yb.size(0)
        scheduler.step()

        train_acc = correct / total * 100
        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}%")

    # Evaluate on test
    model.eval()
    correct, total, all_preds, all_labels = 0, 0, [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            correct += (preds.argmax(dim=1) == yb).sum().item()
            total += yb.size(0)
            all_preds.extend(preds.argmax(dim=1).cpu().numpy())
            all_labels.extend(yb.cpu().numpy())

    test_acc = correct / total * 100
    print(f"Test Accuracy (AfterTreatment classified as class 1): {test_acc:.2f}%")
    print(classification_report(all_labels, all_preds, target_names=["Before", "After"]))

    cm = confusion_matrix(all_labels, all_preds)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Oranges", xticklabels=["Before", "After"], yticklabels=["Before", "After"])
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Classifier Two-Sample Test Confusion Matrix")
    plt.show()

Fetching Before and After data for C2ST...
Treatment: AfterTreatment
  Patient: 11111
    Activity: sit
      Phone Location: DominantWrist
    Activity: walk
      Phone Location: DominantWrist
    Activity: upstairs
      Phone Location: DominantWrist
Treatment: BeforeTreatment
  Patient: 11111
    Activity: sit
      Phone Location: DominantWrist
    Activity: walk
      Phone Location: DominantWrist
    Activity: upstairs
      Phone Location: DominantWrist
Before data chunks: (220, 100, 6), After data chunks: (217, 100, 6)


c:\users\yuanh\appdata\local\programs\python\python38\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100 | Train Loss: 0.3096 | Train Acc: 71.36%
Epoch 2/100 | Train Loss: 0.0038 | Train Acc: 100.00%
Epoch 3/100 | Train Loss: 0.0011 | Train Acc: 100.00%
Epoch 4/100 | Train Loss: 0.0006 | Train Acc: 100.00%
Epoch 5/100 | Train Loss: 0.0004 | Train Acc: 100.00%
Epoch 6/100 | Train Loss: 0.0004 | Train Acc: 100.00%
Epoch 7/100 | Train Loss: 0.0003 | Train Acc: 100.00%
Epoch 8/100 | Train Loss: 0.0003 | Train Acc: 100.00%
Epoch 9/100 | Train Loss: 0.0003 | Train Acc: 100.00%
Epoch 10/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 11/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 12/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 13/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 14/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 15/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 16/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 17/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 18/100 | Train Loss: 0.0002 | Train Acc: 100.00%
Epoch 19/100 | Train

c:\users\yuanh\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\yuanh\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\yuanh\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [6]:
if __name__ == "__main__":
    train_c2st_classifier()